# MCMC tutorial publication results and plotting
Part of the Bayesian neural networks via MCMC: a Python-based tutorial

This notebook contains the code used to generate the results and plots for the paper.

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import glob
from ipywidgets import interact, fixed, widgets
import seaborn as sns
import matplotlib.pyplot as plt

os.chdir('/project')

from publication_results.models.mcmc import MCMC
from publication_results.convergence.convergence import gelman_rubin

## Load the results from the MCMC runs
Load in the results stored in the netcdf files.

In [ ]:
models = ['linear'] #,'bnn'
data = ['Sunspot', 'Abalone', 'Iris', 'Ionosphere']
results_dir = os.path.join('publication_results','results')

results = {mm: {dd: None for dd in data} for mm in models}
for this_model in models:
    mod_results_dir = os.path.join(results_dir,'{}_model'.format(this_model))
    for this_data in data:
        results_files = glob.glob(os.path.join(mod_results_dir,'mcmc_{}_*.nc'.format(this_data)))
        print('Found {} results files for {} model on {} data'.format(len(results_files),this_model,this_data))
        tmp_data = xr.open_mfdataset(results_files, combine='nested',concat_dim='chain')
        # re-ID the chains
        tmp_data.coords['chain'] = np.arange(len(results_files))
        results[this_model][this_data] = tmp_data

In [ ]:
# load the data
x_train = {}
y_train = {}
x_test = {}
y_test = {}

for this_data in data:
    train_data   = np.loadtxt("data/{}/train.txt".format(this_data))
    test_data    = np.loadtxt("data/{}/test.txt".format(this_data))

    x_train[this_data] = train_data[:,:-1]
    y_train[this_data] = train_data[:,-1]
    x_test[this_data] = test_data[:,:-1]
    y_test[this_data] = test_data[:,-1]

# little lookup table
data_case = {'Sunspot':'regression','Abalone':'regression','Iris':'classification','Ionosphere':'classification'}

## Evaluate the model performance
Calculate the model performance metrics for the MCMC samples.

In [ ]:
results_table = []# pd.DataFrame(columns=['train','test','model','data'])
for this_model in models:
    for this_data in data:
        train_dim = results[this_model][this_data].coords['train_idx'].shape[0]
        test_dim = results[this_model][this_data].coords['test_idx'].shape[0]
        train_pred = results[this_model][this_data].train_pred.values.reshape((-1,train_dim))
        test_pred = results[this_model][this_data].test_pred.values.reshape((-1,test_dim))
        
        # Print the train/test RMSE or accuracy in tabular format
        if data_case[this_data] == 'regression':
            train_perf = np.array([MCMC.rmse(train_pred[_,:], y_train[this_data]) for _ in np.arange(train_pred.shape[0])])
            test_perf = np.array([MCMC.rmse(test_pred[_,:], y_test[this_data]) for _ in np.arange(test_pred.shape[0])])

        elif data_case[this_data] == 'classification':
            train_perf = np.array([MCMC.accuracy(train_pred[_,:], y_train[this_data]) for _ in np.arange(train_pred.shape[0])])
            test_perf = np.array([MCMC.accuracy(test_pred[_,:], y_test[this_data]) for _ in np.arange(test_pred.shape[0])])

        results_table.append(
            {
                'train_mean':train_perf.mean(),'test_mean':test_perf.mean(),
                'train_std':train_perf.std(),'test_std':test_perf.std(),
                'model':this_model,'data':this_data
            }
        )
results_table = pd.DataFrame(results_table)
results_table

In [ ]:
# Run the convergence diagnostics
model_r_hat = []
for this_model in models:
    r_hat_collect = []
    for this_data in data:
        print('Convergence diagnostics for {} model on {} data'.format(this_model,this_data))
        # combine the parameters together into a single xarray
        params = results[this_model][this_data].drop(['train_pred','test_pred','train_sim','test_sim','rmse'])
        params = params.drop_vars(['train_idx','test_idx'])
        
        # combine all variables together along a new dimension
        params = params.to_stacked_array(new_dim='param', sample_dims=['chain','samples'])
        
        # compute the Gelman-Rubin statistic
        gr = gelman_rubin(params.values)

        # store results as an additional variable in xarray

        param_names = np.array([_[0] for _ in params.param.values])

        results[this_model][this_data] = results[this_model][this_data].assign_coords(param=param_names)
        results[this_model][this_data] = results[this_model][this_data].assign(rhat=((['param'],gr)))

        r_hat_collect.append(gr)

    r_hat_collect = pd.DataFrame(
        np.concatenate(
            r_hat_collect,
            axis=0
        ),
        columns=['rhat']
    )
    r_hat_collect['model'] = this_model.title()

    model_r_hat.append(r_hat_collect)

model_r_hat = pd.concat(model_r_hat)

In [ ]:
# plot a box and whisker plot with seaborn and the r_hat values
sns.set_context('talk')
fig, ax = plt.subplots(1,1,figsize=(8,6))
sns.boxplot(x='model',y='rhat',data=model_r_hat,ax=ax)
ax.set_ylabel('$\hat{R} values$', labelpad=10)
ax.set_xlabel('Model', labelpad=10)
# y on log scale 
ax.set_yscale('log')
plt.show()


In [ ]:

results[this_model][this_data]

In [ ]:
# Now plot posterior distribution and trace plot for selected parameters

def plot_posterior(this_model, this_data, this_param, chain=0):
    fig, ax = plt.subplots(1,2,figsize=(16,6))

    plot_data = results[this_model][this_data][this_param].sel(chain=chain)

    # plot the posterior distribution
    sns.kdeplot(plot_data.values, ax=ax[0])
    ax[0].set_xlabel(this_param, labelpad=10)
    ax[0].set_ylabel('Density', labelpad=10)
    ax[0].set_title('Posterior distribution')

    # plot the trace plot
    sns.lineplot(x=plot_data.coords['samples'].values,y=plot_data.values,ax=ax[1])
    ax[1].set_xlabel('Samples', labelpad=10)
    ax[1].set_ylabel(this_param, labelpad=10)
    ax[1].set_title('Trace plot')

    fig.tight_layout()
    plt.show()

# Now use interact to plot for given model, data, param

interact(
    plot_posterior, 
    this_model=widgets.Dropdown(options=models, value=models[0]),
    this_data=widgets.Dropdown(options=data, value=data[0]),
    this_param=widgets.SelectionSlider(options=results[models[0]][data[0]].coords['param'].values, value=results[models[0]][data[0]].coords['param'].values[0]),
    chain=widgets.IntSlider(min=0,max=results[models[0]][data[0]].coords['chain'].shape[0]-1,step=1,value=0)
)